In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient
from slugify import slugify
from csv import DictReader
import os
import json
import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from datetime import timedelta
import time
import pickle

In [2]:
import colorsys
def get_N_HexCol(N=20):

    HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in xrange(N)]
    hex_out = []
    for rgb in HSV_tuples:
        rgb = map(lambda x: int(x*255),colorsys.hsv_to_rgb(*rgb))
        hex_out.append("".join(map(lambda x: chr(x).encode('hex'),rgb)))
    return hex_out

In [3]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
    print r
    #print r["data"][0]
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [4]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import GoogleV3
    

my_api_key = os.environ.get('MY_API_KEY', None)
#print my_api_key

def do_geocode(address,G_API_KEY):
    
    
    google_maps = GoogleV3(api_key=G_API_KEY,domain='maps.google.com')
    try:
        return google_maps.geocode(address,"exactly_one=True")
    except GeocoderTimedOut:
        return do_geocode(address)



In [5]:

# credentials
TOPOGRAM_URL = "http://localhost:3000"
#TOPOGRAM_URL = "https://app.topogram.io"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"


In [9]:
with open('./LABEL_graph.obj', 'rb') as handle:
    LABEL_graph=pickle.load(handle)
print len(LABEL_graph[0])
print len(LABEL_graph[1])
print len(LABEL_graph[2])

title = LABEL_graph[0]
title+=" connected Alpha"
my_nodes= LABEL_graph[1]
my_edges= LABEL_graph[2]

15
13864
14271


In [11]:
print title
missing_nodes=[]
to_remove_edges=[]
for node in my_nodes:
    try:
        if not node["data"]["lat"]:
            print node
    except KeyError:
        print node
        
#print map(lambda x: x["data"]["id"],my_nodes)
        
for edge in my_edges:
    if edge["data"]["source"] not in map(lambda x: x["data"]["id"],my_nodes):
        #print edge["data"]["source"]
        missing_nodes.append(edge["data"]["source"])
    if edge["data"]["target"] not in map(lambda x: x["data"]["id"],my_nodes):
        #print edge["data"]["target"]
        to_remove_edges.append(edge["data"]["id"])
print len(missing_nodes)    
print missing_nodes
print "_____________________________#####"
print len(to_remove_edges)
print to_remove_edges
print my_nodes[:10]
print my_edges[:10]



M4P 2 the ST4RZ connected Alpha
0
[]
_____________________________#####
4974
['846b38a4-5295-495d-812d-e3d7f143a3f1=>c51c6914-1820-4d8c-b994-05a8b6b4a6e1', '995428e7-81b6-41dd-bd38-5a7a0ece8ad6=>2b6dcfa9-db2d-4b25-9bb1-9f9a4d5336a3', '995428e7-81b6-41dd-bd38-5a7a0ece8ad6=>53be0cd6-a60b-4236-be5f-467c0453ceb6', '995428e7-81b6-41dd-bd38-5a7a0ece8ad6=>578d18e6-fe1b-4b63-9e26-cf8fa0e26ddb', '995428e7-81b6-41dd-bd38-5a7a0ece8ad6=>5f40c7c4-abde-4504-95d7-193102ef0b78', '995428e7-81b6-41dd-bd38-5a7a0ece8ad6=>7c720ce5-5724-4cf1-93dd-2cd338fae92d', '995428e7-81b6-41dd-bd38-5a7a0ece8ad6=>863f76f2-ada7-432e-8046-bef4e1c11663', '995428e7-81b6-41dd-bd38-5a7a0ece8ad6=>b5d31add-2ca6-45db-b651-282469aa6e57', '995428e7-81b6-41dd-bd38-5a7a0ece8ad6=>c04ecb08-8b95-4901-ba4c-8e247fecb662', '995428e7-81b6-41dd-bd38-5a7a0ece8ad6=>e7a47102-11b8-4008-8541-ed272dbca2fb', '995428e7-81b6-41dd-bd38-5a7a0ece8ad6=>f72ae2f2-0e72-4325-a639-c62079d6045f', '995428e7-81b6-41dd-bd38-5a7a0ece8ad6=>f8a12a01-fe28-44a5-9241-9

In [12]:
i=0
k=0
my_edges_filtered=[]
for edge in my_edges:
    i+=1
    if edge["data"]["id"] in to_remove_edges:
        #print my_edges[i]
        k+=1
        
    else: 
        my_edges_filtered.append(edge)
        #my_edges.pop(i)
        #i-=1
print k
print len(my_edges_filtered)
my_edges=my_edges_filtered

4974
9297


In [13]:
##NOW WE ADD NODE FILTERING TO REDUCE SIZE OF SHOWN NODES
cntr=0
my_nodes_filtered=[]
for node in my_nodes:
    
    if node["data"]["id"]  in map(lambda x: x["data"]["source"],my_edges):
        #print edge["data"]["source"]
        #missing_nodes.append(edge["data"]["source"])
        my_nodes_filtered.append(node)
        
    else:
        if node["data"]["id"]  in map(lambda x: x["data"]["target"],my_edges):
        #print edge["data"]["target"]
            my_nodes_filtered.append(node)
        else:
            #print "removing node not connected",node["data"]["id"]
            cntr+=1
my_nodes=my_nodes_filtered
print len(my_nodes)
print cntr
            
            

4375
9489


In [14]:
# credentials
TOPOGRAM_URL = "http://localhost:3000"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

In [15]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)



ERROR:topogram-client:403 Unauthorized request


> User has already been created.
{u'status': u'success', 'status_code': 200, u'data': {u'authToken': u'5UJbiOT-8ofMr7UAUU2dgBIBt1_I6FRjg-sau5MX2iA', u'userId': u'BxRn4kp4WFL92jW8g'}}


In [17]:

create_topogram(title+" 2", my_nodes, my_edges)

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


Creating topogram 'M4P 2 the ST4RZ connected Alpha 2'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'M4P 2 the ST4RZ connected Alpha 2', u'userId': u'BxRn4kp4WFL92jW8g', u'_id': u'7wrxk3H4a8wMgEP6a', u'slug': u'm4p-2-the-st4rz-connected-alpha-2', u'createdAt': u'2018-05-09T09:23:14.450Z'}, u'statusCode': 200}
topogram ID : 7wrxk3H4a8wMgEP6a
{u'status': u'success', 'status_code': 200, u'data': [u'Smphfm27KBeyeFiNx', u'RWSpTsT7zdX4Nu4xW', u'KTEzgC5ZwbQaK4Qhg', u'ELu22KDL8sktopL3A', u'gP5KNE7SJBNDCWHpk', u'zoFsjmL2vGv2trfkp', u'YWdmAWBLNhwPR5rQX', u'Qe9cWcLW5CHo9EixL', u'XHNysmM7EAw7A9YuK', u'unL4oZ2gSs6zKWfDA', u'5u6eJQh2kBRbGfxiq', u'jnywtqmKsgorSxcPj', u'Rg5ixZTwxibtZ4NQ2', u'Cmcd6AxmvL29qcJ6s', u'J48Zey7wHAgSLdzxf', u'peGQuoqoPyt3zw4Tr', u'7BGM9pQstSXpCBY32', u'Rco9NkpvRXNgqCFSo', u'vwm6eL5PS7WrdhGCN', u'68dFPDboY5YqHa2qx', u'pFM4HpdFybvXtdqzo', u'cKsLDDSdZADw6sBJc', u'5T36nAwRDgXM9QuCe', u'ccPKxbuP2MKXcXiK6', u'yALT

9297 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/7wrxk3H4a8wMgEP6a
